In [1]:
## All of the files for the multirater data need to be available on the DSA DermAnnotator instance
## To make this work, and to make the API less painful, each folder (which is an item) will be annotated
## with additional metadata to make it easier to grab things in a single pull

## There will be a balance of things stored at the folder vs the item level
import os, io, json, girder_client
from pprint import pprint
gc = girder_client.GirderClient(apiUrl="http://dermannotator.org:8080/api/v1")
gc.authenticate(interactive=True)


# localRaterData = {}
# with open("./localRaterData.json","r") as fp:
#     localRaterData = json.load(fp)

Login or email: admin
Password for admin: ········


{'_accessLevel': 2,
 '_id': '59b9553e92ca9a0017e0a4fa',
 '_modelType': 'user',
 'admin': True,
 'created': '2017-09-13T15:56:46.035000+00:00',
 'email': 'histomicstk.admin@kitware.com',
 'emailVerified': True,
 'firstName': 'admin',
 'groupInvites': [],
 'groups': [],
 'lastName': 'admin',
 'login': 'admin',
 'otp': False,
 'public': True,
 'size': 0,
 'status': 'enabled'}

In [2]:
def getSVGJsonData( itemId):
    ## Scan an item ID and grab the files in it... m,akes assumption this only has one file
    for i in gc.get("/item/%s/files" % itemId):
        file = gc.getFile(i['_id'])
        fp = io.BytesIO()
        gc.downloadFile(file['_id'],fp)
        fp.seek(0)
        
        svgJson = json.load(fp)
        return svgJson

def checkFolderForMultiRaterData( folderData):
    ### Given the folder data, which includes the _id , name and metadata folderID, I expect it to have the raw image, an SVG.JSON file, and an overlay image
    ### I need to parse/process these in order to make sure I have all the necessary information for the multirater study
    folderName = folderData['name']
#     print(f"Processing {folderData['name']}")
    
    multiRaterMetaData = { "imageName": folderName, "featuresObservedForImage": [], "markupData": {}, "superpixels_in_mask": [] }
    for i in gc.listItem(folderData['_id']):
        base = os.path.splitext(i['name'])[0]

        ### Generate information for the baseImage
        if (base == folderData['name']):
            ## This should be the main Image for the image
            multiRaterMetaData['mainImage'] = i
            ### I need to get the baseImageHeight and baseImageWidth of the main image to scale things appropriately
            try:
                largeImageInfo = gc.get(f"/item/{i['_id']}/tiles")
            
                multiRaterMetaData['baseImageWidth']  = largeImageInfo['sizeX']
                multiRaterMetaData['baseImageHeight'] = largeImageInfo['sizeY']
            except:
                print(f"No large image info found for {folderName}")
                return None
        ##Look for the SVG Json File
        if ( i['name'].endswith('.svg.json') and folderName in i['name']):
            ### Print Processing the SVG JSON File
            multiRaterMetaData['svgJson'] = getSVGJsonData(i['_id']) 
            
        ## Check for the superpixel png image as well
        if ( i['name'].endswith('_superpixels_v3.0.svg.json') and folderName in i['name']):
            ### Print Processing the SVG JSON File
            multiRaterMetaData['spxImage'] = i 
    return multiRaterMetaData


In [3]:
### MultiRater Root Folder 

multiraterFolder = gc.get("/resource/lookup?path=%s"% "/collection/MultiRater_Viewer_Data/NewDataSet")
### Look for the images/items that are in this folder and see what metadata they have... and add/append if needed

## These are all the image names
for fldr in gc.listFolder(multiraterFolder['_id']):
    #print(fldr['name'],fldr['meta'])
    ## Check folder for metadata which is basically the images and annotation data needed
    multiRaterMetadata = checkFolderForMultiRaterData(fldr)
    
    if not multiRaterMetadata:
        print("Metadata incomplete for %s" % fldr['name'])
    
    gc.addMetadataToFolder(fldr['_id'],multiRaterMetadata)
#     if multiRaterMetadata and fldr['name'] in localRaterData:
#         #multiRaterMetadata['localRaterData'] = localRaterData[fldr['name']]
#         ###For now I am going to copy over the keys from the localRaterData file until I automatically generate them
#         multiRaterMetadata['ratersForCurrentImage'] = localRaterData[fldr['name']]['ratersForCurrentImage']
#         multiRaterMetadata['featuresObservedForImage'] =localRaterData[fldr['name']]['featuresObservedForImage']
#         multiRaterMetadata['markupData'] =localRaterData[fldr['name']]['markupData']
#         multiRaterMetadata['markupData']['superpixels_in_mask'] = localRaterData[fldr['name']]['superpixels_in_mask']
        

KeyboardInterrupt: 